In [ ]:
import copernicusmarine
from pathlib import Path
import xarray as xr
import sys
import os
import datetime
import matplotlib.pyplot as plt
current_dir = os.getcwd()
src_path = Path(current_dir).parent / "src"
sys.path.insert(0, str(src_path))
from satellite_setup import get_satellite_settings

In [ ]:
satellite_dicts = get_satellite_settings()

### Download datasets for each satellite product

In [ ]:
redownload=False
for key, ddict in satellite_dicts.items():
    # go through each of the satellite dicts in turn
    wmts_str = ddict['url']
    # get the dataset id from the WMTS url
    parts = wmts_str.split('/')
    dataset_id = parts[-1]
    dataset_id_parts = dataset_id.split('_')
    if dataset_id_parts[-1][:2] == "20":
        # some kind of dataset version that has to be removed if present
        dataset_id = "_".join(dataset_id_parts[:-1])
    satellite_dicts[key]['dataset_id'] = dataset_id

    matching_datasets = list(Path('.').glob(f'*{dataset_id}*.nc'))
    if matching_datasets and not redownload:
        satellite_dicts[key]['dataset_path'] = matching_datasets[0]
        print(f"dataset already downloaded: {satellite_dicts[key]['dataset_path']}")
        continue
    var_name = ddict['var_name']
    # extract the max date. Set mininum date as 10 days before this to keep downloads manageable
    datetime_max = datetime.datetime.fromisoformat(ddict['datetime_max'])
    datetime_min = datetime_max - datetime.timedelta(days=10)
    # request an area around the SkaMix project area
    response = copernicusmarine.subset(
      dataset_id=dataset_id,
      start_datetime=datetime_min,
      end_datetime=datetime_max,
      minimum_longitude=8,
      maximum_longitude=14,
      minimum_latitude=55,
      maximum_latitude=60,
      maximum_depth=5
    )
    # add the path of downloaded data to the satellite dict
    satellite_dicts[key]['dataset_path'] = response.file_path

### Make some plots

In [ ]:
for key, ddict in satellite_dicts.items():
    ds = xr.open_dataset(ddict['dataset_path'])
    var_name = ddict['var_name']
    fig, ax = plt.subplots(figsize=(10, 8))
    ds_sub  = ds.sel(time=ds.time.max())
    if 'depth' in ds_sub.dims:
        ds_sub = ds_sub.sel(depth=ds_sub.depth.min())
    ds_sub[var_name].plot(ax=ax)
    x = 9.25 if 'forecast' in key else 8.25
    ax.text(x, 59.5, f"{key}: {ddict['dataset_id']}", backgroundcolor='lightgray')